<a href="https://colab.research.google.com/github/andreytsimbalov/News_Classification_and_Vectorization/blob/main/Parser_news_RBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Парсер с RBC новости

Данная программа парсит новости с https://www.rbc.ru/v10/search/ajax/?offset=0&limit=100&material=short_news&dateFrom=01.12.2020&dateTo=31.12.2020 за определенный период 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

md = '/content/drive/My Drive/Colab Notebooks/Diplom/'

In [1]:
import requests
import threading
from bs4 import BeautifulSoup
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import time
import random
import json

%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
days_in_month_dict ={
    1:31,
    2:29,
    3:31,
    4:30,
    5:31,
    6:30,
    7:31,
    8:31,
    9:30,
    10:31,
    11:30,
    12:31
}

number_of_month_dict ={
    1:'01',
    2:'02',
    3:'03',
    4:'04',
    5:'05',
    6:'06',
    7:'07',
    8:'08',
    9:'09',
    10:'10',
    11:'11',
    12:'12'
}

In [12]:
def take_news_item_from_json_item(ji):
  ni = {}
  ni['headline'] = ji['title']
  ni['link'] = ji['fronturl']
  ni['tag'] = ji['category']
  ni['time_int'] = int(ji['publish_date_t']) +10800 #+3 hours on Moscow
  ni['time_in_str'] = ji['publish_date']

  return ni
  
# take_news_item_from_json_item(rrr[0])

In [9]:
def teke_topic_news_dict(month):
  topic_news_dict = []
  start_step = 100
  now_step = start_step
  now_offset = 0
  flag = True

  while flag:
    now_url = 'https://www.rbc.ru/v10/search/ajax/?offset=' + str(now_offset) + \
      '&limit='+str(now_step)+\
      '&material=short_news&dateFrom='+\
      '01.'+number_of_month_dict[month]+\
      '.2020&dateTo='+str(days_in_month_dict[month])+'.'+number_of_month_dict[month]+'.2020'

    requests_item = requests.get(now_url)
    requests_items = json.loads(requests_item.text)['items']

    if requests_items==[]:
      if now_step==1:
        flag = False
        break
      now_step=now_step//10
    else:
      now_news = [take_news_item_from_json_item(_) for _ in requests_items]
      print(f'{now_offset} - {now_step} - {len(now_news)}')
      now_offset+=now_step
      topic_news_dict.extend(now_news)

  return topic_news_dict

In [ ]:
lock = threading.Lock()

def take_text_and_other_tags_news(id, now_link):
  time_delay = time.time()
  
  other_tags = []
  st=''
  flag = True
  iterator_0 = 0
  while flag and iterator_0<10:
    iterator_0+=1

    r = requests.get(now_link)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    try:
      ot = soup.find('div', class_ = 'article__tags__container').find_all('a', class_ = 'article__tags__item')
      other_tags = [_.text for _ in ot]

      sp = soup.find('div', class_ = 'article__text article__text_free').find_all('p', recursive=False)
      for sp_ in sp:
        st+=sp_.text+' '
      st = st.replace('\xa0', ' ')

      tf = st.find('\n')
      if tf!=-1:
        trf = st.rfind('\n')
        st = st[:tf]+st[trf+1:]
    except:
      pass

    if st!='':
      flag = False
    else:
      time.sleep(0.01)

  time_delay = time.time() - time_delay
  global max_time_delay
  global news
  with lock:
      max_time_delay = max(max_time_delay, time_delay)
      news.extend([(id,other_tags, st)])

  if id%100==0:
    print(f'{id}, len_tags: {len(other_tags)}')
  return other_tags, st


# Сбор новостей

In [ ]:
for mount_id in tqdm(range(1,13)):
  print(f'now month {mount_id}')

  topic_news_dict = teke_topic_news_dict(mount_id)

  news = []
  max_time_delay=0
  max_thread_number = 50
  threads_index = set([])


  arr_news_of_time = time.time()

  thrs_main = [threading.Thread(target=take_text_and_other_tags_news, args= (i, tnd['link'],)) 
                for i, tnd in enumerate(topic_news_dict)]

  now_threads = set([])

  iterator = 0

  while iterator<len(thrs_main):
    finished_threads=set([])
    for th in now_threads:
      if not th.isAlive():
        finished_threads.add(th)
    now_threads-=finished_threads
    
    while len(now_threads)<max_thread_number:
      try:
        now_threads.add(thrs_main[iterator])
        thrs_main[iterator].start()
        iterator+=1
        time.sleep(0.03)
      except:
        print(f'iterator: {iterator}')
        break
    time.sleep(0)

  for ths in thrs_main:
    ths.join()

  print(time.time()-arr_news_of_time)
  print(f'max_time_delay {max_time_delay}')
  len(news)

  n_ = np.array([_[2] for _ in news])
  n_count = len(np.where(n_ =='')[0])
  print(f'not found news len - {n_count}')
  

  for n in news:
    topic_news_dict[n[0]]['other_tags'] = n[1]
    topic_news_dict[n[0]]['text'] = n[2]


  data = pd.DataFrame(topic_news_dict)
  data.head()


  data.to_pickle(md+"data/news_rbc_"+ number_of_month_dict[mount_id]+ "_2020")